In [2]:
from transformers import pipeline, AutoTokenizer, BitsAndBytesConfig
import torch
from tqdm import tqdm
import json
import numpy as np
from typing import List
import pandas as pd
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


We want to:
- Find only mcq data
- Sample data to only preserve ~ 7k entries to do the DPO.

In [3]:
# load data from huggingface
from datasets import load_dataset

original_dataset = load_dataset("Tachi67/mnlp_dpo_data")

In [13]:
original_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 24133
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1271
    })
})

In [11]:
def contains_options(example):
    return "Options:" in example['prompt']

mcq_dataset = original_dataset.filter(contains_options)

Filter: 100%|██████████| 1271/1271 [00:00<00:00, 60516.52 examples/s]


In [16]:
mcq_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 12434
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 657
    })
})

In [18]:
# sample 7k rows in the training set
train_dataset = mcq_dataset['train'].shuffle(seed=42).select(range(7000))
test_dataset = mcq_dataset['test']

# concatenate the train and test dataset
from datasets import DatasetDict
mcq_dataset_7k = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [22]:
mcq_dataset_7k.push_to_hub("Tachi67/mnlp_dpo_data_7k")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.01s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Tachi67/mnlp_dpo_data_7k/commit/a292b616de4322d248ea14fef532c5c3c10a7bc6', commit_message='Upload dataset', commit_description='', oid='a292b616de4322d248ea14fef532c5c3c10a7bc6', pr_url=None, pr_revision=None, pr_num=None)